# Распознавание языка текста

<hr>

С.Ю. Папулин (papulin.study@yandex.ru)

### Содержание

- [Статический текст](#Статический-текст)
- [Динамический текст](#Динамический-текст)
    - [Построение модели](#Построение-модели)
    - [Проверка динамического распознавания](#Проверка-динамического-распознавания)

Подключение библиотек:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
import sys
sys.path.insert(0, "../lib/")
from datasets import fetch_20languages

## Статический текст

[Набор данных](https://huggingface.co/datasets/papluca/language-identification)

In [ ]:
# Загрузка данных
dataset = fetch_20languages(return_X_y=True)

# Вывод описания
print(dataset.DESCR)

In [ ]:
df_train = dataset.data['train']
df_train

In [ ]:
# from sklearn.datasets import get_data_home

# # Директория по умолчанию, где хранятся данных
# get_data_home()

In [ ]:
df_train.describe()

In [ ]:
# Количество текстов по каждому классу
df_train['labels'].value_counts()

In [ ]:
# Среднее количество символов в текстах по каждому классу
df_train.groupby('labels')['text'].agg(
    lambda text: text.str.len().mean()
)

In [ ]:
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

In [ ]:
pipeline.fit(df_train['text'], df_train['labels'])

In [ ]:
df_test = dataset.data['test']

In [ ]:
pipeline.score(df_test['text'], df_test['labels'])

## Динамический текст

### Построение модели

In [ ]:
import re

In [ ]:
# Исходные данные
df_train.head()

In [ ]:
# Шаблон для делителя строки на слова
COMPILER = re.compile("\W+", re.UNICODE)

In [ ]:
def split_sentence(lang, text):
    for word in set(COMPILER.split(text)):
        if word:
            yield (lang, word)


def sentence_flat_map(df):
    def fetch_word_lang_pair():
        for i, row in df.iterrows():
            for item in split_sentence(row['labels'], row['text']):
                yield item
    return pd.DataFrame(
        data=fetch_word_lang_pair(), 
        columns=['labels', 'word']
    ).drop_duplicates()


# Формивание датафрейма язык-слово и удаление повторений
df_train_new = sentence_flat_map(df_train)
df_test_new = sentence_flat_map(df_test)

df_train_new

In [ ]:
df_train_new.shape

In [ ]:
INPUT = 'обуч'

print(
    df_train_new[df_train_new['word'].str.contains(INPUT)]\
        .groupby('labels')\
        .count().T
)

In [ ]:
# Априорные вероятности классов
# class_prior=[
#     0.04, 0.04, 0.05, 0.05, 0.1, 0.05, 0.05, 0.04, 0.05, 0.05,
#     0.05, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.05, 0.05, 0.05
# ]
class_prior=[0.05]*20

# Пострение модели классификации
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer='char', ngram_range=(2,4))),
    ('classifier', MultinomialNB(class_prior=class_prior))
])

# Обучение модели
pipeline.fit(df_train_new['word'], df_train_new['labels'])

test_accuracy_on_words = pipeline.score(df_test_new['word'], df_test_new['labels'])
test_accuracy_on_texts = pipeline.score(df_test['text'], df_test['labels'])

# Оценка качества на тестовом множестве
print(f"Accuracy on Test (word-lang) = {test_accuracy_on_words}")

In [ ]:
# Оценка качества на тестовом множестве (из первой задачи)
print(f"Accuracy on Test (text-lang) = {test_accuracy_on_texts}")

In [ ]:
# Словарь
pipeline.named_steps['vectorizer'].vocabulary_

In [ ]:
# Классы
langs = pipeline.named_steps['classifier'].classes_
langs

In [ ]:
INPUT = 'tra'

# Вероятности принадлежности классам для некоторого слова
probs = pipeline.predict_proba([INPUT,])[0]
probs

In [ ]:
print(
    pd.DataFrame(
        data={'prob': probs}, 
        index=langs)\
    .sort_values(by='prob', ascending=False)
)

### Проверка динамического распознавания 

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
def display_prediction(langs, probs):
    """
    Отображение вероятностей по языкам 
    в виде датафрейма.
    """
    print(
        pd.DataFrame(
            data={'prob': probs},
            index=langs
        )\
        .sort_values('prob', ascending=False)\
        .head(10)
    )

In [ ]:
# Ввод текста
text_input = widgets.Text()
display(text_input)

# Вывод результата предсказания
output = widgets.Output()
display(output)


def handle_process_text(sender):
    with output:
        clear_output()
        probs = pipeline.predict_proba([sender.new,])[0]
        langs = pipeline.named_steps['classifier'].classes_
        display_prediction(langs, probs)


# Отслеживание ввода
text_input.observe(handle_process_text, names='value')